# AWS Lambda

AWS Lambda is the Serverless product that AWS introduced at reInvent in November 2014.  Lambda has largely popularized the concept of Serverless and AWS continues to lead in this space.

AWS Lambda allows developers to create and deploy small functions and have the code executed without the need to provision servers.  You pay only for the resources used rather than for idle servers.

**NOTE**: However these functions (FaaS) will often be combined with backend services (BaaS) from AWS or elsewhere which may have more significant costs.  Those backend services include S3 storage, DB storage, API gateways, physical or virtual machines.

**NOTE**: Although FaaS costs themselves are minimal typically with 1mn requests/month free and very small cost per request after that limit is passed, there is a risk of high costs if your application/api goes viral.  It is therefore strongly advised to place rate limits at the API Gateway to prevent runaway costs.

A Lambda function provides a handler which is the entry point for the Lambda written for the chosen runtime such as node.js, python etc.
A Lambda function accepts JSON-formatted input and will usually return the same.

A Lambda function is invoked in response to an event, such as an S3 file upload, a change in a database table, a web request or a scheduled event.

#### Runtimes
The following languages, as well as custom runtimes, are now supported
- C#
- Go
- Java
- Node.js
- Python

## Closed-source

Unfortunately AWS Lambda itself if a proprietary service.

Nevertheless as the leader in this space it is important to understand it's capabilities and limitations.

There are many Open Source tools which allow to deploy to AWS Lambda such as
- Serverless(.com) - allows to deploy to several Serverless platforms
- Chalice (A Python Framework provided by AWS)
- Claudia (A Node.js Framework)

or even to emulate Lambda for testing purposes
- localstack
- docker-lambda (https://github.com/lambci/docker-lambda)

## Setup

To run these exercises you should first have
- installed the awscli package to provide the aws command and also
- configured either
  - the ~/.aws/configure file with your AWS account credentials or
  - created a sourceable ~/.aws/credentials.rc (can be in any location) file

```
> cat ~/.aws/credentials.rc

export AWS_ACCESS_KEY_ID="<your-access-key>"
export AWS_SECRET_ACCESS_KEY="<your-secret-access-key>"
export AWS_DEFAULT_REGION=us-west-1
```

If you have chosen to use an rc file, source it as ```source <your-aws-credentials-rc-file>```, e.g.

In [6]:
. ~/.aws/credentials.rc

cd ~/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda
rm -rf chalice-app scheduled-app

cleanup_functions() {
    echo "-- Functions before:"
    aws lambda list-functions | grep FunctionName

    FNS=$(aws lambda list-functions | grep FunctionName |sed -e 's/.*: //'  -e 's/"//g' -e 's/,//')
    for FN in $FNS; do aws lambda delete-function --function-name $FN; done
    echo "-- Functions after cleanup:"
    aws lambda list-functions | grep FunctionName
}

We can now use the aws cli utility to access lambda commands.

Let's investigate the available commands with ```aws lambda help```

In [7]:
cleanup_functions

-- Functions before:
-- Functions after cleanup:


: 1

In [8]:
aws lambda help 

LAMBDA()                                                              LAMBDA()



NAME
       lambda -

DESCRIPTION
          Overview

       This  is  the AWS Lambda API Reference . The AWS Lambda Developer Guide
       provides additional information. For the service overview, see What  is
       AWS  Lambda  , and for information about how the service works, see AWS
       Lambda: How it Works in the AWS Lambda Developer Guide .

AVAILABLE COMMANDS
       o add-layer-version-permission

       o add-permission

       o create-alias

       o create-event-source-mapping

       o create-function

       o delete-alias

       o delete-event-source-mapping

       o delete-function

       o delete-function-concurrency

       o delete-layer-version

       o get-account-settings

       o get-alias

       o get-event-source-mapping

       o get-function

       o get-function-configuration

       o get-layer-version

       o get-layer-version-policy

       o get-policy

      

Let's see if we have any functions defined already using the ```aws lambda list-functions``` command.
You won't have any functions if you just created your account.

In [9]:
aws lambda list-functions

{
    "Functions": []
}


Note that it is possible to create functions directly using the command ```aws lambda create-function``` but it would be necessary to interact individually with several AWS services.

There exist several tools which facilitate function creation and deployment.

For information about use of ```aws lambda create-function``` refer to these articles
- https://www.tutorialspoint.com/aws_lambda/aws_lambda_creating_and_deploying_using_aws_cli.htm
- https://docs.aws.amazon.com/lambda/latest/dg/API_CreateFunction.html

We will continue using other frameworks.
- Chalice: A Python Serverless Micro-service framework for AWS Lambda
- Claudia: A Node.js Serverless framework for AWS Lambda
- Serverless: A Serverless platform for various platorms (AWS Lambda, Azure Functions, Google CloudFunctions and more)

## Chalice

Chalice is an open source project created by AWS, available here: https://github.com/aws/chalice

Documentation is at https://chalice.readthedocs.io/en/latest/

Chalice allows to
- deploy to a local test server
- deploy to AWS Lambda (as dev or prod stage)
- easily discover the URL of a deployed service
- recuperate '*CloudWatch*' logs of a lambda function (all print function calls go to CloudWatch logs)

Let's see what options are available using ```chalice --help``` command

In [10]:
chalice --help

Usage: chalice [OPTIONS] COMMAND [ARGS]...

Options:
  --version             Show the version and exit.
  --project-dir TEXT    The project directory.  Defaults to CWD
  --debug / --no-debug  Print debug logs to stderr.
  --help                Show this message and exit.

Commands:
  delete
  deploy
  gen-policy
  generate-pipeline  Generate a cloudformation template for a...
  generate-sdk
  invoke             Invoke the deployed lambda function NAME.
  local
  logs
  new-project
  package
  url


## Create a Chalice project

Let's start by creating a new Chalice project:

In [11]:
#cd ..; rm -rf chalice-app

In [12]:
pwd
chalice new-project chalice-app
cd chalice-app
ls -al

/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda
total 20
drwxrwxr-x  3 user1 user1 4096 Jan 26 03:28 .
drwxrwxr-x 10 user1 user1 4096 Jan 26 03:28 ..
drwxrwxr-x  2 user1 user1 4096 Jan 26 03:28 .chalice
-rw-rw-r--  1 user1 user1   37 Jan 26 03:28 .gitignore
-rw-rw-r--  1 user1 user1  736 Jan 26 03:28 app.py
-rw-rw-r--  1 user1 user1    0 Jan 26 03:28 requirements.txt


We see that Chalice had created a template project for us.

Looking at the app.py file we see that we have a Python application to run a REST API server with just one route for '/'.

In [13]:
cp -a app.py app.py.0; cat app.py

from chalice import Chalice

app = Chalice(app_name='chalice-app')


@app.route('/')
def index():
    return {'hello': 'world'}


# The view function above will return {"hello": "world"}
# whenever you make an HTTP GET request to '/'.
#
# Here are a few more examples:
#
# @app.route('/hello/{name}')
# def hello_name(name):
#    # '/hello/james' -> {"hello": "james"}
#    return {'hello': name}
#
# @app.route('/users', methods=['POST'])
# def create_user():
#     # This is the JSON body the user sent in their POST request.
#     user_as_json = app.current_request.json_body
#     # We'll echo the json body back to the user in a 'user' key.
#     return {'user': user_as_json}
#
# See the README documentation for more examples.
#


## Run the function locally

Run the command ```chalice local``` to deploy a local test server

```> chalice local
Serving on http://127.0.0.1:8000
```

In [14]:
http 127.0.0.1:8000

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 17
Content-Type: application/json
Date: Sat, 26 Jan 2019 03:29:14 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "hello": "world"
}



We see that chalice has allowed us to very quickly
- create an application skeleton and to
- test it locally.

Note that Chalice uses @app.route decoration to declare routes, similarly to the Flask micro-framework.

This whole process is greatly simplified compared with deploying Lambda functions directly which would require
- manually creating a zip file of all sources
- uploading the zip file
- creating appropriate policies
- linking the API Gateway to our Lambda function

### Updates

Try making a change to to app.py file and saving it.

Notice that chalice detects the change and redeploys the server to take into account the changes:
```
user1@ip-172-31-21-116:~/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/chalice-app$ chalice local
Serving on http://127.0.0.1:8000
127.0.0.1 - - [25/Jan/2019 00:45:04] "GET / HTTP/1.1" 200 -
Restarting local dev server.
Serving on http://127.0.0.1:8000
Restarting local dev server.
Serving on http://127.0.0.1:8000
```

## Deploying a function to AWS Lambda
Now let's deploy this to AWS Lambda using the ```chalice deploy``` command.

First let's check if any functions are already deployed or not

**NOTE**: So far we have launched a function on a local test server, we have not deployed in to AWS Lambbda.

In [15]:
aws lambda list-functions

{
    "Functions": []
}


then let's look at the options to ```chalice deploy```

In [16]:
chalice deploy --help

Usage: chalice deploy [OPTIONS]

Options:
  --autogen-policy / --no-autogen-policy
                                  Automatically generate IAM policy for app
                                  code.
  --profile TEXT                  Override profile at deploy time.
  --api-gateway-stage TEXT        Name of the API gateway stage to deploy to.
  --stage TEXT                    Name of the Chalice stage to deploy to.
                                  Specifying a new chalice stage will create
                                  an entirely new set of AWS resources.
  --connection-timeout INTEGER    Overrides the default botocore connection
                                  timeout.
  --help                          Show this message and exit.


Now let's deploy our function

In [17]:
chalice deploy

/usr/share/python-wheels/requests-2.18.4-py2.py3-none-any.whl/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
Creating deployment package.
Updating policy for IAM role: chalice-app-dev
Creating lambda function: chalice-app-dev
Creating Rest API
Resources deployed:
  - Lambda ARN: arn:aws:lambda:us-west-1:568285458700:function:chalice-app-dev
  - Rest API URL: https://gxan5eptj8.execute-api.us-west-1.amazonaws.com/api/


We see above the URL on which our API is available, otherwise the ```chalice url``` command provides us with the url.

We can click on that url to open in the browser or access it from the command-line as below.

In [18]:
chalice url

https://gxan5eptj8.execute-api.us-west-1.amazonaws.com/api/


In [19]:
http $(chalice url)

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Connection: keep-alive
Content-Length: 17
Content-Type: application/json
Date: Sat, 26 Jan 2019 03:29:56 GMT
Via: 1.1 34ecd2fc8d142d6781982cb32e3d84bf.cloudfront.net (CloudFront)
X-Amz-Cf-Id: T7UWEHpVgW8Q7SZoO23cIQzCzS4QZiYzOV6Dl6dJLL5BDuVThsRHng==
X-Amzn-Trace-Id: Root=1-5c4bd433-c8d0a66daeb62bf3ff72f42b;Sampled=0
X-Cache: Miss from cloudfront
x-amz-apigw-id: UF4YHH8zSK4Ffaw=
x-amzn-RequestId: a6972411-211a-11e9-9265-b7cb00312a65

{
    "hello": "world"
}



In just a few commands we've been able to create a project template, run a local test server and deploy our skeleton app to AWS Lambda.

Chalice greatly simplifies the deployment of Python functions.

Note that we can list deployed functions using ```aws lambda list-functions```.

Note also that the function name is chalice-app-dev, by default chalice deploys in '*dev*' mode.

In [20]:
aws lambda list-functions

{
    "Functions": [
        {
            "FunctionName": "chalice-app-dev",
            "FunctionArn": "arn:aws:lambda:us-west-1:568285458700:function:chalice-app-dev",
            "Runtime": "python3.6",
            "Role": "arn:aws:iam::568285458700:role/chalice-app-dev",
            "Handler": "app.app",
            "CodeSize": 10518,
            "Description": "",
            "Timeout": 60,
            "MemorySize": 128,
            "LastModified": "2019-01-26T03:29:43.294+0000",
            "CodeSha256": "Uba16QADvw213FT3PyeqJhSxrDsDnym6JnMqYWZXysU=",
            "Version": "$LATEST",
            "VpcConfig": {
                "SubnetIds": [],
                "SecurityGroupIds": [],
                "VpcId": ""
            },
            "Environment": {
                "Variables": {}
            },
            "TracingConfig": {
                "Mode": "PassThrough"
            },
            "RevisionId": "23d15802-cfa7-4a72-8a61-68fc131d700e"
        }
    ]
}


In [21]:
cat > app.py <<EOF

from chalice import Chalice

app = Chalice(app_name='chalice-app')

@app.route('/')
def index():
    print("Looks like we got a request!") ## Should appear in CloudWatch logs
    return {'hello': 'world'}
    
EOF

Now let's redeploy our function and make a request to it

In [22]:
chalice deploy

/usr/share/python-wheels/requests-2.18.4-py2.py3-none-any.whl/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
Creating deployment package.
Updating policy for IAM role: chalice-app-dev
Updating lambda function: chalice-app-dev
Updating rest API
Resources deployed:
  - Lambda ARN: arn:aws:lambda:us-west-1:568285458700:function:chalice-app-dev
  - Rest API URL: https://gxan5eptj8.execute-api.us-west-1.amazonaws.com/api/


In [23]:
http $(chalice url)

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Connection: keep-alive
Content-Length: 17
Content-Type: application/json
Date: Sat, 26 Jan 2019 03:30:43 GMT
Via: 1.1 e75f5fedf935ed2a4a49debe6c78d172.cloudfront.net (CloudFront)
X-Amz-Cf-Id: Fh_dlKn3cOKAx_mcoMLfVl97SMMzm2wOc5Dr9U7QAEJj6lm07OhavQ==
X-Amzn-Trace-Id: Root=1-5c4bd463-4d76411edd93fea6b81f166c;Sampled=0
X-Cache: Miss from cloudfront
x-amz-apigw-id: UF4feElUSK4FlTA=
x-amzn-RequestId: c2b2e128-211a-11e9-8589-d30094675486

{
    "hello": "world"
}



In [24]:
chalice logs --include-lambda-messages | grep Looks

2019-01-25 23:01:06.688000 27caab Looks like we got a request!
2019-01-26 02:52:51.463000 ebd3d5 Looks like we got a request!
2019-01-26 03:20:40.767000 cec580 Looks like we got a request!


In [25]:
cleanup_functions

-- Functions before:
            "FunctionName": "chalice-app-dev",
-- Functions after cleanup:


: 1

# Scheduled events

In [26]:
aws lambda list-functions 

{
    "Functions": []
}


In [27]:
cd ~/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda


chalice new-project scheduled-app

In [28]:
cd scheduled-app
ls -al 

total 20
drwxrwxr-x  3 user1 user1 4096 Jan 26 03:31 .
drwxrwxr-x 11 user1 user1 4096 Jan 26 03:31 ..
drwxrwxr-x  2 user1 user1 4096 Jan 26 03:31 .chalice
-rw-rw-r--  1 user1 user1   37 Jan 26 03:31 .gitignore
-rw-rw-r--  1 user1 user1  738 Jan 26 03:31 app.py
-rw-rw-r--  1 user1 user1    0 Jan 26 03:31 requirements.txt


In [29]:
pwd
cat app.py

/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/scheduled-app
from chalice import Chalice

app = Chalice(app_name='scheduled-app')


@app.route('/')
def index():
    return {'hello': 'world'}


# The view function above will return {"hello": "world"}
# whenever you make an HTTP GET request to '/'.
#
# Here are a few more examples:
#
# @app.route('/hello/{name}')
# def hello_name(name):
#    # '/hello/james' -> {"hello": "james"}
#    return {'hello': name}
#
# @app.route('/users', methods=['POST'])
# def create_user():
#     # This is the JSON body the user sent in their POST request.
#     user_as_json = app.current_request.json_body
#     # We'll echo the json body back to the user in a 'user' key.
#     return {'user': user_as_json}
#
# See the README documentation for more examples.
#


In [30]:
cat > app.py <<EOF

from chalice import Chalice, Rate

app = Chalice(app_name="scheduled-app")
app.debug = True

# Automatically runs every 1 minutes
@app.schedule(Rate(1, unit=Rate.MINUTES))
def periodic_task(event):
    print("Scheduled[1 min]")
    return {"hello": "world"}
EOF

In [31]:
pwd
aws lambda list-functions

/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/scheduled-app
{
    "Functions": []
}


In [32]:
chalice deploy

/usr/share/python-wheels/requests-2.18.4-py2.py3-none-any.whl/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
Creating deployment package.
Creating IAM role: scheduled-app-dev
Creating lambda function: scheduled-app-dev-periodic_task
Resources deployed:
  - Lambda ARN: arn:aws:lambda:us-west-1:568285458700:function:scheduled-app-dev-periodic_task


In [43]:
chalice logs --include-lambda-messages

In [44]:
aws lambda list-functions

{
    "Functions": [
        {
            "FunctionName": "scheduled-app-dev-periodic_task",
            "FunctionArn": "arn:aws:lambda:us-west-1:568285458700:function:scheduled-app-dev-periodic_task",
            "Runtime": "python3.6",
            "Role": "arn:aws:iam::568285458700:role/scheduled-app-dev",
            "Handler": "app.periodic_task",
            "CodeSize": 10309,
            "Description": "",
            "Timeout": 60,
            "MemorySize": 128,
            "LastModified": "2019-01-26T03:33:52.685+0000",
            "CodeSha256": "uh3k06OKlHnGH1dTkfoSX0YTDp4Q6zT9rUWW3AWqB+w=",
            "Version": "$LATEST",
            "VpcConfig": {
                "SubnetIds": [],
                "SecurityGroupIds": [],
                "VpcId": ""
            },
            "Environment": {
                "Variables": {}
            },
            "TracingConfig": {
                "Mode": "PassThrough"
            },
            "RevisionId": "85b4aac2-b384-4bad-8120-4c1

In [48]:
chalice logs --include-lambda-messages

2019-01-25 00:00:11.996000 44d41c START RequestId: 969cdc86-aa85-4719-bf77-aff5d863a8a9 Version: $LATEST
2019-01-25 00:00:11.996000 44d41c END RequestId: 969cdc86-aa85-4719-bf77-aff5d863a8a9
2019-01-25 00:00:11.996000 44d41c REPORT RequestId: 969cdc86-aa85-4719-bf77-aff5d863a8a9	Duration: 0.57 ms	Billed Duration: 100 ms 	Memory Size: 128 MB	Max Memory Used: 24 MB
2019-01-25 00:07:22.639000 44d41c START RequestId: 1a0a3cdc-b023-4d69-98f5-a145f0e478ee Version: $LATEST
2019-01-25 00:07:22.709000 44d41c END RequestId: 1a0a3cdc-b023-4d69-98f5-a145f0e478ee
2019-01-25 00:07:22.709000 44d41c REPORT RequestId: 1a0a3cdc-b023-4d69-98f5-a145f0e478ee	Duration: 29.26 ms	Billed Duration: 100 ms 	Memory Size: 128 MB	Max Memory Used: 24 MB
2019-01-25 00:28:18.733000 44d41c START RequestId: 6ba33c1e-d1b2-4da5-8933-4933a5ebfff3 Version: $LATEST
2019-01-25 00:28:18.750000 44d41c END RequestId: 6ba33c1e-d1b2-4da5-8933-4933a5ebfff3
2019-01-25 00:28:18.751000 44d41c REPORT RequestId: 6ba33c1e-d1b2-4da5-8933-

2019-01-26 00:32:19.330000 b6528d END RequestId: 5533a0b2-d2f8-468c-81dc-2429073b668a
2019-01-26 00:32:19.330000 b6528d REPORT RequestId: 5533a0b2-d2f8-468c-81dc-2429073b668a	Duration: 634.91 ms	Billed Duration: 700 ms 	Memory Size: 128 MB	Max Memory Used: 37 MB
2019-01-26 00:34:33.414000 a8ddcb START RequestId: bce9a152-cb73-4261-9bec-3c0d10582899 Version: $LATEST
2019-01-26 00:34:33.415000 a8ddcb s3objects: SO FAR SO GOOD
2019-01-26 00:34:33.415000 a8ddcb s3objects: <{'query_params': None, 'headers': {'accept': '*/*', 'accept-encoding': 'gzip, deflate', 'cloudfront-forwarded-proto': 'https', 'cloudfront-is-desktop-viewer': 'true', 'cloudfront-is-mobile-viewer': 'false', 'cloudfront-is-smarttv-viewer': 'false', 'cloudfront-is-tablet-viewer': 'false', 'cloudfront-viewer-country': 'US', 'host': 'jlmq3ygdu1.execute-api.us-west-1.amazonaws.com', 'user-agent': 'HTTPie/0.9.8', 'via': '1.1 5c2ff4ca1e447265402af29264e83497.cloudfront.net (CloudFront)', 'x-amz-cf-id': 'mQCX2b5PHSGu4WVl-i1aUxzX

2019-01-26 00:37:56.009000 a8ddcb raise JSONDecodeError("Expecting value", s, err.value) from None
2019-01-26 00:37:56.009000 a8ddcb json.decoder.JSONDecodeError: Expecting value: line 2 column 1 (char 1)
2019-01-26 00:37:56.009000 a8ddcb END RequestId: 91b39421-4222-4d9a-8381-a3ac5ac88572
2019-01-26 00:37:56.009000 a8ddcb REPORT RequestId: 91b39421-4222-4d9a-8381-a3ac5ac88572	Duration: 555.22 ms	Billed Duration: 600 ms 	Memory Size: 128 MB	Max Memory Used: 38 MB
2019-01-26 00:39:25.506000 882ee8 START RequestId: 33e782d0-3f4f-42a3-a976-24acf9df7954 Version: $LATEST
2019-01-26 00:39:25.506000 882ee8 s3objects: SO FAR SO GOOD -----------------------------------------------------------------
2019-01-26 00:39:25.506000 882ee8 s3objects: <{'query_params': None, 'headers': {'accept': '*/*', 'accept-encoding': 'gzip, deflate', 'cloudfront-forwarded-proto': 'https', 'cloudfront-is-desktop-viewer': 'true', 'cloudfront-is-mobile-viewer': 'false', 'cloudfront-is-smarttv-viewer': 'false', 'cloudf

2019-01-26 00:46:09.358000 830682 GET s3://mjbright-uploads/test
2019-01-26 00:46:10.009000 830682 END RequestId: 7a774729-e5d8-4279-97ca-67ef720318d2
2019-01-26 00:46:10.009000 830682 REPORT RequestId: 7a774729-e5d8-4279-97ca-67ef720318d2	Duration: 632.71 ms	Billed Duration: 700 ms 	Memory Size: 128 MB	Max Memory Used: 37 MB
2019-01-26 01:03:44.795000 189dfc START RequestId: 049afcfb-6aca-4f20-9bc9-022e42bbd3a5 Version: $LATEST
2019-01-26 01:03:44.809000 189dfc s3text: ----------------------------------------------------------------- <{'query_params': None, 'headers': {'accept': '*/*', 'accept-encoding': 'gzip, deflate', 'cloudfront-forwarded-proto': 'https', 'cloudfront-is-desktop-viewer': 'true', 'cloudfront-is-mobile-viewer': 'false', 'cloudfront-is-smarttv-viewer': 'false', 'cloudfront-is-tablet-viewer': 'false', 'cloudfront-viewer-country': 'US', 'host': 'jlmq3ygdu1.execute-api.us-west-1.amazonaws.com', 'user-agent': 'HTTPie/0.9.8', 'via': '1.1 e2af8a85927835558866752f53562ecd.cl

2019-01-26 01:04:01.431000 189dfc s3text: ----------------------------------------------------------------- <{'query_params': None, 'headers': {'accept': '*/*', 'accept-encoding': 'gzip, deflate', 'cloudfront-forwarded-proto': 'https', 'cloudfront-is-desktop-viewer': 'true', 'cloudfront-is-mobile-viewer': 'false', 'cloudfront-is-smarttv-viewer': 'false', 'cloudfront-is-tablet-viewer': 'false', 'cloudfront-viewer-country': 'US', 'host': 'jlmq3ygdu1.execute-api.us-west-1.amazonaws.com', 'user-agent': 'HTTPie/0.9.8', 'via': '1.1 49c80a47c1441dd194a8337982f1cd7e.cloudfront.net (CloudFront)', 'x-amz-cf-id': 'THP5eRWQ4cEezLBbXU-2vT4VzPDF4-mCFcMy6JMGOQTUd367pcvL4w==', 'x-amzn-trace-id': 'Root=1-5c4bb201-b56313b28f6fdc1ce3892232', 'x-forwarded-for': '54.67.51.193, 54.239.134.14', 'x-forwarded-port': '443', 'x-forwarded-proto': 'https'}, 'uri_params': {'key': 'test'}, 'method': 'GET', 'context': {'resourceId': 'ezap0f', 'resourcePath': '/text/{key}', 'httpMethod': 'GET', 'extendedRequestId': 'U

2019-01-26 01:04:02.437000 189dfc s3json: ----------------------------------------------------------------- <{'query_params': None, 'headers': {'accept': '*/*', 'accept-encoding': 'gzip, deflate', 'cloudfront-forwarded-proto': 'https', 'cloudfront-is-desktop-viewer': 'true', 'cloudfront-is-mobile-viewer': 'false', 'cloudfront-is-smarttv-viewer': 'false', 'cloudfront-is-tablet-viewer': 'false', 'cloudfront-viewer-country': 'US', 'host': 'jlmq3ygdu1.execute-api.us-west-1.amazonaws.com', 'user-agent': 'HTTPie/0.9.8', 'via': '1.1 8008015354a3ca72f56c382a1d1cfe9f.cloudfront.net (CloudFront)', 'x-amz-cf-id': 'Fl_4Y6lsAmuU7vr5sIZ7jXR0N3k0V6d2fnm9cb2FBowui4L_K5pqpQ==', 'x-amzn-trace-id': 'Root=1-5c4bb202-550f0d68a4b72c38b4559ae8', 'x-forwarded-for': '54.67.51.193, 54.239.134.14', 'x-forwarded-port': '443', 'x-forwarded-proto': 'https'}, 'uri_params': {'key': 'test'}, 'method': 'GET', 'context': {'resourceId': '1ay49m', 'resourcePath': '/json/{key}', 'httpMethod': 'GET', 'extendedRequestId': 'U

2019-01-26 01:05:14.449000 189dfc {   'context': {   'accountId': '568285458700',
2019-01-26 01:05:14.449000 189dfc 'apiId': 'jlmq3ygdu1',
2019-01-26 01:05:14.449000 189dfc 'domainName': 'jlmq3ygdu1.execute-api.us-west-1.amazonaws.com',
2019-01-26 01:05:14.449000 189dfc 'domainPrefix': 'jlmq3ygdu1',
2019-01-26 01:05:14.469000 189dfc 'extendedRequestId': 'UFjLoHZfyK4FlIg=',
2019-01-26 01:05:14.469000 189dfc 'httpMethod': 'PUT',
2019-01-26 01:05:14.469000 189dfc 'identity': {   'accessKey': None,
2019-01-26 01:05:14.469000 189dfc 'accountId': None,
2019-01-26 01:05:14.469000 189dfc 'caller': None,
2019-01-26 01:05:14.469000 189dfc 'cognitoAuthenticationProvider': None,
2019-01-26 01:05:14.469000 189dfc 'cognitoAuthenticationType': None,
2019-01-26 01:05:14.469000 189dfc 'cognitoIdentityId': None,
2019-01-26 01:05:14.469000 189dfc 'cognitoIdentityPoolId': None,
2019-01-26 01:05:14.469000 189dfc 'sourceIp': '54.67.51.193',
2019-01-26 01:05:14.469000 189dfc 'user': None,
2019-01-26 01:05:14

## Cleanup: deleting the function deployment

We can cleanup by deleting the function which we deployed

In [49]:
#aws lambda delete-function --function-name chalice-app-dev
#aws lambda list-functions

# Function capabilities

Let's now investigate how we can treat different routes, requests, responses with Chalice.

## Returning custom headers, status_code
Until now we returned directly a *json* structure but we can control the type of response we return e.g. text/plain instead of json, or other headers.  It is also possible to set an error status code.

In the below example we
- return a 200 OK status_code
- return a plaintext 'hello world!' body, whilst setting content-type to 'text/plain' in the headers
- we also add a custom 'X-Debug' header

In [50]:
cd ~/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda

cd chalice-app
pwd

/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/chalice-app


In [51]:
cat > app.py << EOF
from chalice import Chalice, Response

app = Chalice(app_name='chalice-app')


@app.route('/')
def index():

    custom_headers = {
        'Content-Type': 'text/plain',
        'X-Debug': 'some debug information'
    }

    return Response(body='hello world!',
                  status_code=200,
                  headers=custom_headers)
EOF

In [52]:
http 127.0.0.1:8000

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 12
Content-Type: text/plain
Date: Sat, 26 Jan 2019 03:38:50 GMT
Server: BaseHTTP/0.6 Python/3.6.7
X-Debug: some debug information

hello world!



## Implementing a REST API

We can also use the decorators to specify parametrized url patterns to specify parameters to a request, and also specify GET, PUT, POST, DELETE requests allowing to implement a REST API.

In the below example we add new routes:
- GET /hello/{name} allowing to greet a specified named person
- POST /users       allowing to create a user (*)

**Note**: this is only notional creation of a user as we have no persistence across function calls.  We would need to add some persistent backend storage for that.

In [53]:
cat > app.py <<EOF
from chalice import Chalice

app = Chalice(app_name='helloworld')

@app.route('/')
def index():
    return {'hello': 'world'}

# The view function above will return {"hello": "world"}
# whenever you make an HTTP GET request to '/'.
#
# Here are a few more examples:
#
@app.route('/hello/{name}')
def hello_name(name):
   # '/hello/james' -> {"hello": "james"}
   return {'hello': name}

@app.route('/users', methods=['POST'])
def create_user():
    # This is the JSON body the user sent in their POST request.
    user_as_json = app.current_request.json_body
    # We'll echo the json body back to the user in a 'user' key.
    return {'user': user_as_json}

EOF

We can make a **GET** request to **/hello/bob** to make a greeting to 'bob':

In [54]:
http :8000/hello/bob

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 15
Content-Type: application/json
Date: Sat, 26 Jan 2019 03:39:03 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "hello": "bob"
}



We can make a **POST** request to /users to create a new user - this function returns a json definition of this user although that definition is not persisted by the function itself.

In [55]:
echo '"testuser"' | http POST :8000/users

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 19
Content-Type: application/json
Date: Sat, 26 Jan 2019 03:39:08 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "user": "testuser"
}



In [56]:
echo '["testuser", {"fullname": "My full name"}]' | http POST :8000/users

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 49
Content-Type: application/json
Date: Sat, 26 Jan 2019 03:39:09 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "user": [
        "testuser",
        {
            "fullname": "My full name"
        }
    ]
}



## Accessing a datastore

For the moment we have no persistent datastore, for the moment we use a poor man's datastore, a hardcoded dictionary in our code !!

**Note**: This is not so ridiculous.  We could imagine some scenarii where the Serverless function is used with an immutable database, maybe even deployed as a Unikernel.  It could be an infrequently modified store such as a DNS database where a new appliction is created when DNS changes are to be made.

Let's implement something vaguely DNS-like ...

In [57]:
cat > app.py <<EOF
from chalice import Chalice

app = Chalice(app_name='bindservice')

'''
Note: Enabling app.debug gives much better error message for missing key than:
    { "Code": "ChaliceViewError",
      "Message": "ChaliceViewError: An internal server error occurred."
    }
'''
app.debug = True

HOST_IPv4_MAPPING = {
    'localhost': '127.0.0.1',
    'google.com': ['172.217.5.110', '216.58.195.68'],
}

# Yes this could be calcuated from HOST_IPv4_MAPPING but better to have an immutable static copy created when the applicaton is created.
IPv4_HOST_MAPPING = {
    '127.0.0.1': 'localhost',
    '216.58.195.68': 'google.com',
    '172.217.5.110': 'google.com',
}


'''
* Query database for the host with specified IPv4 address
'''
@app.route('/host/{ip}')
def get_host(ip):
    return {'host': IPv4_HOST_MAPPING[ip]}

'''
* Query database for the  IPv4 addresses for the specified host
'''
@app.route('/ips/{host}')
def get_host(host):
    return {'ips': HOST_IPv4_MAPPING[host]}

EOF

### Querying on IPv4 address

In [58]:
http 127.0.0.1:8000/host/127.0.0.1

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 20
Content-Type: application/json
Date: Sat, 26 Jan 2019 03:39:27 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "host": "localhost"
}



In [59]:
http 127.0.0.1:8000/host/172.217.5.110

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 21
Content-Type: application/json
Date: Sat, 26 Jan 2019 03:39:29 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "host": "google.com"
}



### Querying on hostname

In [60]:
http 127.0.0.1:8000/ips/google.com

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 200 OK
Content-Length: 41
Content-Type: application/json
Date: Sat, 26 Jan 2019 03:39:32 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "ips": [
        "172.217.5.110",
        "216.58.195.68"
    ]
}



### But we need error handling

But our current implementation doesn't look very smart if we look up an unknown item

In [61]:
http 127.0.0.1:8000/ips/UNKNOWN.com

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 500 Internal Server Error
Content-Length: 372
Content-Type: text/plain
Date: Sat, 26 Jan 2019 03:39:36 GMT
Server: BaseHTTP/0.6 Python/3.6.7

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/chalice/app.py", line 731, in _get_view_function_response
    response = view_function(**function_args)
  File "/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/chalice-app/app.py", line 38, in get_host
    return {'ips': HOST_IPv4_MAPPING[host]}
KeyError: 'UNKNOWN.com'



### Error handling

We can handle error cases such as catching an unknown ip address or hostname

In [64]:
cat > app.py <<EOF
from chalice import Chalice
from chalice import NotFoundError

app = Chalice(app_name='bindservice')

'''
  NOTE: Can use different error codes:
    * BadRequestError   - return a status code of 400
    * UnauthorizedError - return a status code of 401
    * ForbiddenError    - return a status code of 403
    * NotFoundError     - return a status code of 404
    * ConflictError     - return a status code of 409
    * UnprocessableEntityError - return a status code of 422
    * TooManyRequestsError - return a status code of 429
    * ChaliceViewError  - return a status code of 500
'''

app.debug = True

HOST_IPv4_MAPPING = {
    'localhost': '127.0.0.1',
    'google.com': ['172.217.5.110', '216.58.195.68']
}

# Yes this could be calcuated from HOST_IPv4_MAPPING but better to have an immutable static copy created when the applicaton is created.
IPv4_HOST_MAPPING = {
    '127.0.0.1': 'localhost',
    '216.58.195.68': 'google.com',
    '172.217.5.110': 'google.com'
}


'''
* Query database for the host with specified IPv4 address
'''
@app.route('/host/{ip}')
def get_host(ip):
    try:
        host = {'host':  IPv4_HOST_MAPPING[ip]} 
    except KeyError:
        raise NotFoundError("Unknown ip '%s'" % ( ip ))
        
    return host

'''
* Query database for the  IPv4 addresses for the specified host
'''
@app.route('/ips/{host}')
def get_host(host):
    try:
        ips = {'ips': HOST_IPv4_MAPPING[host]}
    except KeyError:
        raise NotFoundError("Unknown host '%s'" % ( host ))
        
    return ips

EOF

Now when we rerun some error cases we get a cleaner response:

In [65]:
#http 127.0.0.1:8000/host/127.0.0.1
#http 127.0.0.1:8000/host/172.217.5.110
http 127.0.0.1:8000/host/172.217.5.255 # UNKNOWN
#http 127.0.0.1:8000/ips/google.com
http 127.0.0.1:8000/ips/UNKNOWN.com  # UNKNOWN

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 404 Not Found
Content-Length: 78
Content-Type: application/json
Date: Sat, 26 Jan 2019 03:43:31 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "Code": "NotFoundError",
    "Message": "NotFoundError: Unknown ip '172.217.5.255'"
}

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 404 Not Found
Content-Length: 78
Content-Type: application/json
Date: Sat, 26 Jan 2019 03:43:31 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "Code": "NotFoundError",
    "Message": "NotFoundError: Unknown host 'UNKNOWN.com'"
}



## Think of some scenarii

### Think about some use cases for this functionality.

### Is it clear why it is interesting to do this as a serverless function?

Remember that in this example we are not configuring a server to host our database and so we are very light on resource usage and we pay only for what we use.

If we use a persistent backing store, S3 file storage or a database, then we wil pay for that usage but likely we will be sharing an existing file storage or database server and so we stil have a low cost solution where our front end application doesn't require a server to itself.

We also get automatic scaling of our function if there is a lot of traffic from the network

### Think of some scenario where we use all http/REST methods GET, PUT, POST, DELETE

### So far we've used an embedded immutable database - what could we do with a persistent storage backend ?


# More Reading



# Cleanup



In [ ]:
cleanup_functions

In [ ]:
aws s3 rm s3://mjbright-static-site --recursive
aws s3 rb s3://mjbright-static-site

It's also possible to remoce the bucket directly using the ```--force``` option:
    ```aws s3 rb --force s3://mjbright-static-site```